In [ ]:
from __future__ import print_function

import numpy as np
import keras.backend as K
from keras.engine.topology import Layer
from keras.layers import Conv2D, Reshape, Lambda
from keras.activations import softmax


def margin_loss(y_true, y_pred, 
    m_plus=0.9, m_minus=0.1, down_weighting=0.5):
    
    L = y_true*K.square(K.maximum(0.0, m_plus-y_pred)) + down_weighting*(1-y_true)*K.square(K.maximum(0.0, y_pred-m_minus))

    return K.mean(K.sum(L, 1))



In [ ]:

import numpy as np
import pandas as pd
import pylab as plt
import cv2
import os
from sklearn.metrics import confusion_matrix
import mlxtend.plotting
from tqdm import tqdm
from IPython.display import clear_output
from keras.utils.np_utils import to_categorical
from tensorflow.keras.models import  Model
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.optimizers import Adam,Adamax
from tensorflow.keras.layers import Conv2D, SeparableConv2D, Dense, Dropout, concatenate, MaxPooling2D, BatchNormalization, GlobalAveragePooling2D, Input
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer
from tensorflow.keras import activations
from tensorflow.keras import utils

from tensorflow.keras.layers import *



In [ ]:
IMG_SIZE=50


In [ ]:
train=np.load('E:/thorax_data.npy')


In [ ]:
train=train[:]


In [ ]:

X_NMF_G = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
X_NMF_A = np.array([i[0] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
X = np.array([i[1] for i in train]).reshape(-1,IMG_SIZE,IMG_SIZE,1)
YD = np.array([i[2] for i in train])
YG = np.array([i[3] for i in train])
YA = np.array([i[4] for i in train])


In [ ]:
from sklearn.model_selection import train_test_split
XG, XGt, YG1, YGt1 = train_test_split(X_NMF_G, YG, test_size=0.1, random_state=42)
XA, XAt, YA1, YAt1 = train_test_split(X_NMF_A, YA, test_size=0.1, random_state=42)
X, Xt, Y1, Yt1 = train_test_split(X, YD, test_size=0.1, random_state=42)

In [ ]:
X=X/255
Xt=Xt/255
XG=XG/255
XA=XA/255
XGt=XGt/255
XAt=XAt/255


In [ ]:
from sklearn.utils import class_weight

In [ ]:
from keras import backend as K
def weighted_categorical_crossentropy(weights):
    """
    A weighted version of keras.objectives.categorical_crossentropy
    
    Variables:
        weights: numpy array of shape (C,) where C is the number of classes
    
    Usage:
        weights = np.array([0.5,2,10]) # Class one at 0.5, class 2 twice the normal weights, class 3 10x.
        loss = weighted_categorical_crossentropy(weights)
        model.compile(loss=loss,optimizer='adam')
    """
    
    weights = K.variable(weights)
        
    def loss(y_true, y_pred):
        # scale predictions so that the class probas of each sample sum to 1
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        # clip to prevent NaN's and Inf's
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        # calc
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    
    return loss

In [ ]:
from keras import backend as K

def dice_coef(y_true, y_pred, smooth=1):
    """
    Dice = (2*|X & Y|)/ (|X|+ |Y|)
         =  2*sum(|A*B|)/(sum(A^2)+sum(B^2))
    ref: https://arxiv.org/pdf/1606.04797v1.pdf
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    return (2. * intersection + smooth) / (K.sum(K.square(y_true),-1) + K.sum(K.square(y_pred),-1)+ smooth)

def dice_coef_loss(y_true, y_pred):
    return (1-dice_coef(y_true, y_pred))
def dice_coef_loss2(y_true, y_pred):
    
    return 0.8*(1-dice_coef(y_true, y_pred))

In [ ]:
def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1 - dice_coef(y_true, y_pred)


def tversky(y_true, y_pred, smooth=1, alpha=0.7):
    y_true_pos = K.flatten(y_true)
    y_pred_pos = K.flatten(y_pred)
    true_pos = K.sum(y_true_pos * y_pred_pos)
    false_neg = K.sum(y_true_pos * (1 - y_pred_pos))
    false_pos = K.sum((1 - y_true_pos) * y_pred_pos)
    return (true_pos + smooth) / (true_pos + alpha * false_neg + (1 - alpha) * false_pos + smooth)


def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)


def focal_tversky_loss(y_true, y_pred, gamma=0.75):
    tv = tversky(y_true, y_pred)
    return K.pow((1 - tv), gamma)

In [ ]:
import tensorflow as tf
gamma=2.
alpha=4.

gamma = float(gamma)
alpha = float(alpha)

def focal_loss_fixed(y_true, y_pred):
    """Focal loss for multi-classification
    FL(p_t)=-alpha(1-p_t)^{gamma}ln(p_t)
    Notice: y_pred is probability after softmax
    gradient is d(Fl)/d(p_t) not d(Fl)/d(x) as described in paper
    d(Fl)/d(p_t) * [p_t(1-p_t)] = d(Fl)/d(x)
    Focal Loss for Dense Object Detection
    https://arxiv.org/abs/1708.02002

    Arguments:
    y_true {tensor} -- ground truth labels, shape of [batch_size, num_cls]
            y_pred {tensor} -- model's output, shape of [batch_size, num_cls]

    Keyword Arguments:
            gamma {float} -- (default: {2.0})
            alpha {float} -- (default: {4.0})

        Returns:
            [tensor] -- loss.
    """
    epsilon = 1.e-9
    y_true = tf.convert_to_tensor(y_true, tf.float32)
    y_pred = tf.convert_to_tensor(y_pred, tf.float32)

    model_out = tf.add(y_pred, epsilon)
    ce = tf.multiply(y_true, -tf.log(model_out))
    weight = tf.multiply(y_true, tf.pow(tf.subtract(1., model_out), gamma))
    fl = tf.multiply(alpha, tf.multiply(weight, ce))
    reduced_fl = tf.reduce_max(fl, axis=1)
    return tf.reduce_mean(reduced_fl)
    

In [ ]:
from keras import backend as K

def jaccard_distance_loss(y_true, y_pred, smooth=100):
    """
    Jaccard = (|X & Y|)/ (|X|+ |Y| - |X & Y|)
            = sum(|A*B|)/(sum(|A|)+sum(|B|)-sum(|A*B|))
    
    The jaccard distance loss is usefull for unbalanced datasets. This has been
    shifted so it converges on 0 and is smoothed to avoid exploding or disapearing
    gradient.
    
    Ref: https://en.wikipedia.org/wiki/Jaccard_index
    
    @url: https://gist.github.com/wassname/f1452b748efcbeb4cb9b1d059dce6f96
    @author: wassname
    """
    intersection = K.sum(K.abs(y_true * y_pred), axis=-1)
    sum_ = K.sum(K.abs(y_true) + K.abs(y_pred), axis=-1)
    jac = (intersection + smooth) / (sum_ - intersection + smooth)
    return (1 - jac) * smooth

In [ ]:
class PlotLearning(Callback):
    def on_train_begin(self, logs={}):
        self.i = 0
        self.x = []
        self.losses = []
        self.val_losses = []
        self.acc = []
        self.val_acc = []
        self.fig = plt.figure()

        self.logs = []

    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(logs)
        self.x.append(self.i)
        self.losses.append(logs.get('loss'))
        self.val_losses.append(logs.get('val_loss'))
        self.acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
        self.i += 1
        f, (ax1, ax2) = plt.subplots(1, 2, sharex=True)

        clear_output(wait=True)

        ax1.set_yscale('Log')
        ax1.plot(self.x, self.losses, label="loss")
        ax1.plot(self.x, self.val_losses, label="val_loss")
        ax1.legend()

        ax2.plot(self.x, self.acc, label="acc")
        ax2.plot(self.x, self.val_acc, label="val_acc")
        ax2.legend()

        plt.show()

from keras.optimizers import rmsprop, adamax
plot = PlotLearning()

In [ ]:
def scheduler(epoch, lr):
    if epoch < 10:
        return lr
    else:
        return lr * tensorflow.math.exp(-0.1)
callback1 = tensorflow.keras.callbacks.LearningRateScheduler(scheduler)


In [ ]:
from tensorflow.keras.initializers import glorot_uniform
initializer = glorot_uniform()
input_layer = Input((50,50, 1))

###########################################################
#Convolutional Stream1
###########################################################

Conv11 = Conv2D(32, (3,3), activation='relu')(input_layer)#CB11
Conv12 = Conv2D(32, (3,3), activation='relu',padding='same')(Conv11)
Conv12 = BatchNormalization()(Conv12)
Conv12 = Dropout(0.5)(Conv12)
CMaxpool11 = MaxPooling2D((3,3))(Conv12)

Conv13 = Conv2D(64, (3,3), activation='relu',padding='same')(CMaxpool11)#CB12
Conv14 = Conv2D(64, (3,3), activation='relu',padding='same')(Conv13)
Conv14 = BatchNormalization()(Conv14)
Conv14 = Dropout(0.5)(Conv14)
CMaxpool12 = MaxPooling2D((3,3))(Conv14)

Conv15 = Conv2D(128, (3,3), activation='relu',padding='same')(CMaxpool12)#CB13
Conv16 = Conv2D(128, (3,3), activation='relu',padding='same')(Conv15)
Conv16 = BatchNormalization()(Conv16)
Conv16 = Dropout(0.5)(Conv16)
CMaxpool13 = MaxPooling2D((2,2))(Conv16)


Conv21 = Conv2D(32, (5,5), activation='relu')(input_layer)#CB11
Conv22 = Conv2D(32, (5,5), activation='relu',padding='same')(Conv21)
Conv22 = BatchNormalization()(Conv22)
Conv22 = Dropout(0.5)(Conv22)
CMaxpool21 = MaxPooling2D((3,3))(Conv22)

Conv23 = Conv2D(64, (5,5), activation='relu',padding='same')(CMaxpool21)#CB12
Conv24 = Conv2D(64, (5,5), activation='relu',padding='same')(Conv23)
Conv24 = BatchNormalization()(Conv24)
Conv24 = Dropout(0.5)(Conv24)
CMaxpool22 = MaxPooling2D((3,3))(Conv24)

Conv25 = Conv2D(128, (5,5), activation='relu',padding='same')(CMaxpool22)#CB13
Conv26 = Conv2D(128, (5,5), activation='relu',padding='same')(Conv25)
Conv26 = BatchNormalization()(Conv26)
Conv26 = Dropout(0.5)(Conv26)
CMaxpool23 = MaxPooling2D((2,2))(Conv26)

Allnet=concatenate([CMaxpool13,CMaxpool23])

AllNet = Flatten()(Allnet)
AllNet = Dense(1024, activation='relu')(AllNet)
AllNet = Dropout(0.5)(AllNet)

predictions = Dense(2, activation="softmax",name='Gender')(AllNet)

model = Model(inputs=input_layer, outputs=predictions)
model.compile(loss=margin_loss,
              optimizer=Adam(lr=0.00001,beta_1=0.9, beta_2=0.999,epsilon=0.00001), metrics = ['accuracy'])

 
model.summary()



In [ ]:


from tensorflow.keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=1, min_lr=0.0)

In [ ]:
history=model.fit(XG,YG1,epochs=1,validation_split=0.1,batch_size=16,shuffle=True,callbacks=[plot,reduce_lr])

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 18})
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
scores = model.evaluate(XGt, YGt1)
print(model.metrics_names, scores)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))



In [ ]:
#plot confusion_matrix
pred = model.predict(XGt)
pred = np.argmax(pred,axis = 1)
y_true = np.argmax(YGt1,axis = 1)
CM = confusion_matrix(y_true, pred)
fig, ax = mlxtend.plotting.plot_confusion_matrix(conf_mat=CM, figsize=(5, 5))
plt.show()

In [ ]:
y_score = model.predict(XGt)
y_test=YGt1

In [ ]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
n_classes=2
# Plot linewidth.
lw = 1.5
lw2=1
Names=['Fmale', 'Male']
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])




colors = cycle(['black', 'red'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='{0} (area = {1:0.2f})'
             ''.format(Names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw2)
plt.xlim([-0.02, 1.0])
plt.ylim([-0.02, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, 1.05), ncol=2, fancybox=False, shadow=False,prop={'size': 13})

plt.show()
#plt.savefig('F:/Older than 7 jul 20/Chest_G_A_D/AUC_Dis1450NMReal1.png', shadow=True, dpi=1000,transparent=True,bbox_inches="tight",pad_inches=0)


In [ ]:
model.layers[20]


In [ ]:
for i in range(6):
    model.layers[i].trainable = False

x = model.layers[20].output

x = Flatten()(x)
x = Dense(1024)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output1 = Dense(4, activation = 'softmax',name='Age')(x)
model = Model(model.input,output1)

# In[11]:


model.compile(loss=margin_loss,
              optimizer=Adam(lr=0.00001,beta_1=0.9, beta_2=0.999,epsilon=0.00001), metrics = ['accuracy'])

In [ ]:
history=model.fit(XA,YA1,epochs=1,validation_split=0.1,batch_size=16,shuffle=True,callbacks=[plot,reduce_lr])

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 18})
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
scores = model.evaluate(XAt, YAt1)
print(model.metrics_names, scores)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))



In [ ]:
#plot confusion_matrix
pred = model.predict(XAt)
pred = np.argmax(pred,axis = 1)
y_true = np.argmax(YAt1,axis = 1)
CM = confusion_matrix(y_true, pred)
fig, ax = mlxtend.plotting.plot_confusion_matrix(conf_mat=CM, figsize=(5, 5))
plt.show()

In [ ]:
y_score = model.predict(XAt)
y_test=YAt1

In [ ]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
n_classes=4
# Plot linewidth.
lw = 1.5
lw2=1
Names=['A1', 'A2','A3','A4']
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])




colors = cycle(['black', 'red','green','blue'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='{0} (area = {1:0.2f})'
             ''.format(Names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw2)
plt.xlim([-0.02, 1.0])
plt.ylim([-0.02, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, 1.05), ncol=2, fancybox=False, shadow=False,prop={'size': 13})

plt.show()


In [ ]:
model.layers[10]


In [ ]:
for i in range(6):
    model.layers[i].trainable = False

x = model.layers[10].output

x = Flatten()(x)
x = Dense(1024)(x)
x = Activation('relu')(x)
x = BatchNormalization()(x)
x = Dropout(0.5)(x)
output1 = Dense(15, activation = 'softmax',name='dis')(x)

model = Model(model.input,output1)

# In[11]:


model.compile(loss=margin_loss,
              optimizer=Adam(lr=0.00001,beta_1=0.9, beta_2=0.999,epsilon=0.00001), metrics = ['accuracy'])

In [ ]:
history=model.fit(X,Y1,epochs=1,validation_split=0.1,batch_size=16,shuffle=True,callbacks=[plot,reduce_lr])

In [ ]:
from matplotlib import pyplot as plt
plt.rcParams["font.family"] = "Times New Roman"
plt.rcParams.update({'font.size': 18})
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
scores = model.evaluate(Xt, Yt1)
print(model.metrics_names, scores)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1] * 100))



In [ ]:
#plot confusion_matrix
pred = model.predict(Xt)
pred = np.argmax(pred,axis = 1)
y_true = np.argmax(Yt1,axis = 1)
CM = confusion_matrix(y_true, pred)
fig, ax = mlxtend.plotting.plot_confusion_matrix(conf_mat=CM, figsize=(15, 15))
plt.show()

In [ ]:
y_score = model.predict(Xt)
y_test=Yt1

In [ ]:
import numpy as np
from scipy import interp
import matplotlib.pyplot as plt
from itertools import cycle
from sklearn.metrics import roc_curve, auc
n_classes=15
# Plot linewidth.
lw = 1.5
lw2=1
Names=['D1_AT', 'D2_CM','D3_CD','D4_ED','D5_EF','D6_EM','D7_FB','D8_HN','D9_IF','D10_MS','D11_ND','D12_NF','D13_PN','D14_PT','D15_PT']
# Compute ROC curve and ROC area for each class
fpr = dict()
tpr = dict()
roc_auc = dict()
for i in range(n_classes):
    fpr[i], tpr[i], _ = roc_curve(y_test[:, i], y_score[:, i])
    roc_auc[i] = auc(fpr[i], tpr[i])

# Compute micro-average ROC curve and ROC area
fpr["micro"], tpr["micro"], _ = roc_curve(y_test.ravel(), y_score.ravel())
roc_auc["micro"] = auc(fpr["micro"], tpr["micro"])

# Compute macro-average ROC curve and ROC area

# First aggregate all false positive rates
all_fpr = np.unique(np.concatenate([fpr[i] for i in range(n_classes)]))

# Then interpolate all ROC curves at this points
mean_tpr = np.zeros_like(all_fpr)
for i in range(n_classes):
    mean_tpr += interp(all_fpr, fpr[i], tpr[i])

# Finally average it and compute AUC
mean_tpr /= n_classes

fpr["macro"] = all_fpr
tpr["macro"] = mean_tpr
roc_auc["macro"] = auc(fpr["macro"], tpr["macro"])




colors = cycle(['pink','black', 'red','green', 'orange','blue','purple', 'cyan','brown', 'pink','olive', 'wheat','darkorange','darkblue','lime', 'gray'])
for i, color in zip(range(n_classes), colors):
    plt.plot(fpr[i], tpr[i], color=color, lw=lw,
             label='{0} (area = {1:0.2f})'
             ''.format(Names[i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--', lw=lw2)
plt.xlim([-0.02, 1.0])
plt.ylim([-0.02, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('')
plt.legend(loc='lower center', bbox_to_anchor=(0.5, 1.05), ncol=2, fancybox=False, shadow=False,prop={'size': 13})

plt.show()
